In [1]:
# all needed imports
import csv
import time
import pprint
import random
import os
import matplotlib.pyplot as plt
import matplotlib.lines as plt_lines
#p=pprint.PrettyPrinter(width=1)
#p.pprint({data:uh})

In [2]:
# read data from txt or csv file
def read_data(file_path, nb_cities_to_visit):
  '''
    function 'read_data' takes two parameters:
      -file_path: path to the file to read
      -nb_cities_to_visit: <= the size of the given array
    and it has no return

    it initialize the global array 'costs'
  '''
  global costs
  with open(f'{file_path}') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',' if file_path.endswith('.csv') else '\t')
      costs, counter = [], 0
      for line in csv_reader:
        if counter<nb_cities_to_visit:
          costs.append(list(map(int, line))[:nb_cities_to_visit])
        else: break
        counter+=1
  # p=pprint.PrettyPrinter()
  # p.pprint(costs)

In [3]:
def tsp_dynamic_programming(first_city: int):
  '''
    function 'tsp_dynamic_programming' takes two parameters:
      -costs: 2 dimensional list, contains the cost betwing two cities i and j
        *example: cost(i, j) = data[i][j]
      -first_city: the starting city
    and it returns two variables:
      -best_tour: one dimonsional list contains the best tour
      -optimal_cost: the optimal cost
  '''
  def memoize(f):
      '''
        function 'memorize' takes one parameter:
          -f: the function which we want to memorize its previous call returns
      '''
      memo_dict = {}
      def memo_func(*args):
          if args not in memo_dict:
              memo_dict[args] = f(*args)
          return memo_dict[args]
      memo_func.clear = lambda: memo_dict.clear()
      return memo_func
  ff=first_city
  @memoize
  def rec_tsp_solving(target_city, inter_cities):
      assert target_city not in inter_cities,"ERROR: target city can not be in intermediate cities"
      if inter_cities:
          return min((costs[lc][target_city] + rec_tsp_solving(lc, inter_cities - set([lc]))[0], lc)
                      for lc in inter_cities)
      else:
        return (costs[ff][target_city], target_city)

  best_tour, costs_length = [], len(costs)
  init_city = first_city
  inter_cities = frozenset([i for i in range(costs_length) if i!=first_city])
  optimal_cost=-1 #unknown
  while True:
      l_cost, before_last_city = rec_tsp_solving(first_city, inter_cities)
      if optimal_cost==-1: optimal_cost=l_cost
      if before_last_city == first_city: break
      best_tour.insert(0, before_last_city)
      first_city = before_last_city
      inter_cities -= frozenset([before_last_city])

  return [init_city,*best_tour,init_city],optimal_cost

**Metaheuristic GVNS (General Variable Neighborhood Search)**

Minimization object function

In [4]:
def f(x: list):
    cost = 0
    for i in range(1,len(x)):
        cost += costs[x[i - 1]][x[i]]
    return cost

Initialize an initial random path that GVNS works based on

In [5]:
def initilize(first_city):
  init_path=[i for i in range(len(costs)) if i!=first_city]
  random.shuffle(init_path)
  return [first_city, *init_path, first_city]

Shaking

In [6]:
def neighborhood(x, e = 1):
  N = []
  bound = int(len(x) - e)
  for i in range(bound):
    nx = x.copy()
    nx[i],nx[i + e] = nx[i + e],nx[i]
    N.append(nx)
  return N

In [7]:
def shake(x, k):
  N = neighborhood(x, e = k)
  init_x = random.choice(N)
  return init_x

Changing neighborhood

In [8]:
def change_neighborhood(x, bx, k):
  if f(x)<f(bx):
      bx, k = x, 1
  else:
      k += 1
  return bx, k

Local VND

In [9]:
def FirstImprovement(bx, k):
  while True:
    N = neighborhood(bx, k)
    x = bx
    for i in N:
      if f(i)<f(x):
        x = i
        break
    if f(x)>=f(bx): break
    else: bx = x
  return bx

In [10]:
#Variable Neighbourhood Descent Algorithm
def VND(bx, N):
  k = 0
  while k<N:
      x = FirstImprovement(bx, k)
      bx, k = change_neighborhood(x, bx, k)
  return bx

In [11]:
def GVNS(bx, execution_time, max_k=3, max_l=2, first_city=0):
  start_time = time.time()
  while time.time()-start_time < execution_time*60:
      k = 1
      while k <= max_k:
          x_ = VND(shake(bx, k), max_l)
          bx, k = change_neighborhood(bx, x_, k)
          bx=[first_city, *[i for i in bx if i!=first_city], first_city]
  return bx,f(bx)

Draw Simulation of found results

In [12]:
def targetPlot(optimal_path:list, optimal_distance:int, coords:list=None, cities_name:list=None):
    lenght=len(costs)
    if not coords:
      coords=[[random.randint(lenght,3*lenght),random.randint(lenght,3*lenght)] for i in range(lenght)]
    print(f"Used coordinates of cities (generated if there are not given):")
    p=pprint.PrettyPrinter()
    p.pprint(coords)
    cmap = plt.get_cmap('Set2')
    colors = [cmap(i) for i in [round(random.random(), 8) for i in range(lenght)]]

    # Figure build & display markers
    f=plt.figure(figsize=(13, 12))
    ax = f.add_subplot(111)
    plt.scatter([i[0] for i in coords], [i[1] for i in coords], marker='o', c=colors, s=2*lenght)
    
    # Legend build
    markers = []
    labels = [str(i) for i in range(lenght)] if not cities_name else cities_name
    for i in range(lenght):
        markers.append(plt_lines.Line2D([0], [0], linestyle='None', marker="o", markersize=20, markerfacecolor=colors[i]))
    lgd = plt.legend(markers, labels, numpoints=1, bbox_to_anchor=(1, 1)) #(1.17, 0.5)

    # Plot descriptive text
    textstr =f'''Number of cities : {lenght}
Optimal distance: {optimal_distance}
Start city: {optimal_path[0] if not cities_name else cities_name[optimal_path[0]]}'''
    props = {"boxstyle":'round', "facecolor":'wheat', "alpha":0.5}
    plt.text(0.05, 0.98, textstr,transform = ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)

    # Trace the target
    start_node = optimal_path[0]
    for i in range(lenght):
        start_pos = coords[start_node-1]
        next_node = optimal_path[i+1]
        end_pos = coords[next_node-1]
        ax.annotate("",
                xy=start_pos, xycoords='data',
                xytext=end_pos, textcoords='data',
                arrowprops={'arrowstyle':"<|-",'connectionstyle':"arc3"}
                )
        start_node = next_node
    plt.axis('equal')
    plt.show()

Main part of the application

In [ ]:
while True:
  try:
    file_path=input("Insert (.txt, .csv) file path: ")
    nb_cities_to_visit=int(input("Number of cities to visit: "))
    read_data(file_path, nb_cities_to_visit)
    break
  except IOError:
    print(f"ERROR: file '{file_path}' can't be found")
  except ValueError:
    print(f"ERROR: '{nb_cities_to_visit}' it's not a number")

# # while True:
os.system("clear")
print("Menu".center(30, "-"))
print('''
1- Solve With Dynamic Programming Approach (Exact Approach)
2- Solve With GVNS Approach (Metaheuristic Approach)
3- exit the application
''')
try:
  choise=int(input("Make your choise: "))
  f_city=int(input("First city to visit: "))
  if choise==1:
    t0=time.time()
    optimal_path,optimal_distance=tsp_dynamic_programming(f_city)
    t1=time.time()
  elif choise==2:
    execution_time=int(input("Execution time (in minutes): "))
    t0=time.time()
    optimal_path,optimal_distance=GVNS(initilize(f_city), execution_time, first_city=f_city)
    t1=time.time()
  else: exit()
  print(f'''
  Optimal path is: {optimal_path}
  Optimal distance is: {optimal_distance}
  Execution time is: {t1-t0}
  ''')
  targetPlot(optimal_path,optimal_distance)
except ValueError:
  print("ERROR: Invalid option, please try again")